In [1]:
import cv2
import polars
import os
import numpy as np
import polars as pl
from IPython.display import display

In [2]:
data_path = "data"
image_data_path = os.path.join(data_path, "image")
generated_image_data_path = os.path.join(image_data_path, "generated")

In [3]:
image_classes = os.listdir(generated_image_data_path)
image_paths = {image_class: [] for image_class in image_classes}
for image_class in image_classes:
    image_path = os.path.join(generated_image_data_path, image_class)
    image_paths[image_class] = sorted(os.listdir(image_path))

In [4]:
images = {image_class: [] for image_class in image_classes}
for image_class, image_list in image_paths.items():
    images[image_class] = [
        cv2.imread(
            os.path.join(
                generated_image_data_path,
                image_class,
                image_name,
            )
        )
        for image_name in image_list
    ]

In [5]:
def extract_features(image):
    blue  = image[:, :, 0]
    green = image[:, :, 1]
    red   = image[:, :, 2]
    return{
        "blue_mean": np.mean(blue),
        "blue_std": np.std(blue),
        "green_mean": np.mean(green),
        "green_std": np.std(green),
        "red_mean": np.mean(red),
        "red_std": np.std(red),
    }

In [7]:
rows = [
    {**extract_features(img), "class": img_class}
    for img_class, img_matrices in images.items()
    for img in img_matrices
]
df = pl.DataFrame(rows)
df.write_csv(os.path.join(data_path, "tabular", "feature_extraction.csv"))
display(df)

blue_mean,blue_std,green_mean,green_std,red_mean,red_std,class
f64,f64,f64,f64,f64,f64,str
75.192955,95.64547,92.357155,105.600454,99.321049,112.099186,"""banana"""
169.612247,75.303959,187.843922,51.601779,194.192684,47.84109,"""banana"""
157.572319,73.423733,193.686344,48.08909,210.02092,47.674383,"""banana"""
150.701717,65.034955,179.173409,59.486491,182.290588,76.773148,"""banana"""
51.750942,52.189217,111.73209,77.884898,154.641254,90.336133,"""banana"""
…,…,…,…,…,…,…
84.33025,55.283538,140.394249,72.3681,196.035351,68.273515,"""tomato"""
104.729927,72.279619,131.530888,69.865022,194.016422,33.864521,"""tomato"""
114.713585,75.533635,150.429588,75.018363,203.775719,54.988048,"""tomato"""
